# import

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt


# load

In [ ]:
df = pd.read_csv("./chat.csv")
text = "\n".join(df["message"].to_list())
len(text), text[3400:4500]

# custom tokenizer

In [ ]:
def find_top_pair(tokens):
    token_counts = {}
    for i in range(len(tokens)-1):
        token_counts[(tokens[i], tokens[i+1])] = token_counts.get((tokens[i], tokens[i+1]), 0) + 1
    sorted_tokens = list(sorted(token_counts.items(), key=lambda item: item[1], reverse=True))
    if sorted_tokens[0][1] == 1:
        return (), 0
    return sorted_tokens[0]

def merge(tokens_to_merge, pair, new_index):
    new_tokens = []
    i = 0
    while i<len(tokens_to_merge):
        if i<len(tokens_to_merge)-1 and tokens_to_merge[i] == pair[0] and tokens_to_merge[i+1] == pair[1]:
            new_tokens.append(new_index)
            i+=2
        else:
            new_tokens.append(tokens_to_merge[i])
            i+=1
    return new_tokens

tokens = list(text.encode('utf8'))
merges = {}
desired_vocab_size = 400
paired_tokens = list(tokens)
new_tokens_count = desired_vocab_size - 256
merge_counts = []
for i in range(new_tokens_count):
    top_pair, count = find_top_pair(paired_tokens)
    new_index = 256 + i
    if not bool(top_pair):
        print("no more top pairs, exiting")
        break
    paired_tokens = merge(paired_tokens, top_pair, new_index)
    merges[top_pair] = new_index
    merge_counts.append(count)
print(paired_tokens[:500])

In [ ]:
print(f"compresion: paired tokens / tokens {len(tokens) / len(paired_tokens):.2f}x")
plt.plot(merge_counts, marker='o')
plt.title('Plot of Sorted Numbers')
plt.xlabel('Index')
plt.ylabel('Value')
plt.grid(True)
plt.show()

In [ ]:
vocab = {index: bytes([index]) for index in range(256)}
for (p0, p1), index in merges.items():
    vocab[index] = vocab[p0] + vocab[p1]

def decode(indices):
    tokens = b"".join(vocab[index] for index in indices)
    return tokens.decode("utf8", errors="replace")
decode(paired_tokens[350:400])


def encode(text :str):
    tokens = list(text.encode("utf8"))
    for (p0, p1), replace_index in merges.items():
        i = 0
        while i<len(tokens):
            if i<len(tokens)-1 and tokens[i] == p0 and tokens[i+1] == p1:
                tokens[i] = replace_index
                del tokens[i + 1]
                i+=2
            else:
                i+=1
    return tokens

assert encode(decode(paired_tokens[350:400])) == paired_tokens[350:400]
decode(encode(decode(paired_tokens[350:400])))